## **Install Required Libraries**

In [1]:
!pip install rasa spacy scikit-learn pandas numpy
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 32.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 7.6 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 20.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the

## **Importing Libraries and Downloading Dataset**



*   Dataset Information:
    - shuyangli94/food-com-recipes-and-user-interactions from Kaggle
    - URL for dataset is in output for a lookat
    - Size of around 800mbish



In [2]:
import pandas as pd
import numpy as np
import json
import os
import re
from sklearn.model_selection import train_test_split
import spacy

# Let's download a recipe dataset from Kaggle
# You'll need to upload your kaggle.json credentials file first
# or use this cell to upload it
from google.colab import files
files.upload()  # Upload your kaggle.json here

# Configure Kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset - Food.com recipes dataset
!kaggle datasets download -d shuyangli94/food-com-recipes-and-user-interactions
!unzip food-com-recipes-and-user-interactions.zip

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/shuyangli94/food-com-recipes-and-user-interactions
License(s): copyright-authors
Archive:  food-com-recipes-and-user-interactions.zip
  inflating: PP_recipes.csv          
  inflating: PP_users.csv            
  inflating: RAW_interactions.csv    
  inflating: RAW_recipes.csv         
  inflating: ingr_map.pkl            
  inflating: interactions_test.csv   
  inflating: interactions_train.csv  
  inflating: interactions_validation.csv  


### Focusing on certain dishes such as :


*   Pasta
*   Spaghetti
*   Lasagna



### PreProcessing of Dataset:

In [3]:
# Load the dataset
recipes_df = pd.read_csv('RAW_recipes.csv')

# Display basic information
print(recipes_df.shape)
print(recipes_df.columns)
recipes_df.head()

# Clean and preprocess the data
recipes_df['ingredients'] = recipes_df['ingredients'].apply(eval)  # Convert string representation of list to actual list
recipes_df['steps'] = recipes_df['steps'].apply(eval)  # Convert string representation of list to actual list

# Extract recipes for specific dishes - let's focus on pasta dishes as an example
pasta_recipes = recipes_df[recipes_df['name'].str.contains('pasta|spaghetti|lasagna', case=False, na=False)]
print(f"Number of pasta recipes: {len(pasta_recipes)}")

# Create a smaller dataset for our project
recipe_data = pasta_recipes[['name', 'id', 'minutes', 'ingredients', 'steps', 'description']]
recipe_data.to_csv('pasta_recipes.csv', index=False)
recipe_data.head()

(231637, 12)
Index(['name', 'id', 'minutes', 'contributor_id', 'submitted', 'tags',
       'nutrition', 'n_steps', 'steps', 'description', 'ingredients',
       'n_ingredients'],
      dtype='object')
Number of pasta recipes: 6643


,name,id,minutes,ingredients,steps,description
44,italian sandwich pasta salad,33606,25,"[tri-color spiral pasta, dill pickles, ripe ol...","[cook pasta and set aside, place onions , dill...",this is a fun salad that uses the same ingredi...
51,killer lasagna,53402,90,"[italian sausage, ground beef, garlic, dried b...",[brown the sausage and ground meat and drain a...,this was originally my mom's recipe and used t...
54,leftovers spaghetti sauce,100870,40,"[stewed tomatoes, prego spaghetti sauce, garli...",[i had previously browned and drained the grou...,i love eating ground beef in my spaghetti sauc...
57,make that chicken dance salsa pasta,32169,30,"[tomatoes, garlic, onion, button mushrooms, ho...",[heat oil in a frying pan and saut the onions ...,i made this up the other night after i started...
60,mexican pasta,60219,25,"[fettuccine pasta, cheddar cheese soup, milk, ...","[boil pasta as directed on package, drain and ...",made this as a side dish for mexican meatballs...


In [4]:
# Create a directory for our Rasa project
!mkdir -p recipe_bot

# Change to the project directory
%cd recipe_bot

# Initialize Rasa project
!rasa init --no-prompt

/content/recipe_bot
/bin/bash: line 1: rasa: command not found


### Configuring rasa NLU and its config yml file

In [7]:
# Modify the domain.yml file
with open('domain.yml', 'w') as f:
    f.write('''
version: "3.1"

intents:
  - greet
  - goodbye
  - affirm
  - deny
  - ask_recipe
  - ask_ingredients
  - ask_cooking_time
  - ask_cooking_steps
  - inform_dish
  - inform_ingredients
  - inform_dietary_restrictions

entities:
  - dish
  - ingredient
  - dietary_restriction

slots:
  dish:
    type: text
    influence_conversation: true
    mappings:
    - type: from_entity
      entity: dish
  ingredients:
    type: list
    influence_conversation: true
    mappings:
    - type: from_entity
      entity: ingredient
  dietary_restriction:
    type: text
    influence_conversation: true
    mappings:
    - type: from_entity
      entity: dietary_restriction

responses:
  utter_greet:
  - text: "Hello! I'm your recipe assistant. What would you like to cook today?"

  utter_goodbye:
  - text: "Goodbye! Happy cooking!"

  utter_ask_dish:
  - text: "What dish would you like to make?"

  utter_ask_ingredients:
  - text: "Do you have any specific ingredients you'd like to use?"

  utter_ask_dietary_restrictions:
  - text: "Do you have any dietary restrictions I should know about?"

  utter_recipe_not_found:
  - text: "I'm sorry, I couldn't find a recipe for {dish}. Would you like to try something else?"

session_config:
  session_expiration_time: 60
  carry_over_slots_to_new_session: true

actions:
  - action_provide_recipe
  - action_list_ingredients
  - action_explain_steps
  - action_suggest_alternatives
''')

### Configuring rasa nlu's training data

In [9]:
# Create the data directory if it doesn't exist
!mkdir -p data

# Create NLU training data
with open('data/nlu.yml', 'w') as f:
    f.write('''
version: "3.1"

nlu:
- intent: greet
  examples: |
    - hey
    - hello
    - hi
    - good morning
    - good evening
    - hey there

- intent: goodbye
  examples: |
    - bye
    - goodbye
    - see you around
    - see you later

- intent: affirm
  examples: |
    - yes
    - indeed
    - of course
    - that sounds good
    - correct

- intent: deny
  examples: |
    - no
    - never
    - I don't think so
    - don't like that
    - no way
    - not really

- intent: ask_recipe
  examples: |
    - I want to make [pasta](dish)
    - How do I make [lasagna](dish)?
    - Recipe for [spaghetti bolognese](dish)
    - I need a recipe for [fettuccine alfredo](dish)
    - Show me how to cook [pasta carbonara](dish)
    - Can you give me a recipe for [pasta primavera](dish)?
    - I'd like to prepare [ravioli](dish)
    - How to make [macaroni and cheese](dish)?

- intent: ask_ingredients
  examples: |
    - What ingredients do I need for [pasta](dish)?
    - List the ingredients for [spaghetti](dish)
    - What do I need to make [lasagna](dish)?
    - Ingredients for [fettuccine](dish)?
    - What should I buy to make [pasta carbonara](dish)?

- intent: ask_cooking_time
  examples: |
    - How long does it take to make [pasta](dish)?
    - Cooking time for [spaghetti](dish)?
    - How much time do I need for [lasagna](dish)?
    - How long should I cook [ravioli](dish)?

- intent: ask_cooking_steps
  examples: |
    - How do I make [pasta](dish) step by step?
    - What are the steps to cook [spaghetti](dish)?
    - Explain how to make [lasagna](dish)
    - What's the process for making [fettuccine](dish)?
    - Guide me through making [pasta carbonara](dish)

- intent: inform_dish
  examples: |
    - [pasta](dish)
    - [spaghetti](dish)
    - [lasagna](dish)
    - [fettuccine alfredo](dish)
    - [pasta carbonara](dish)
    - [ravioli](dish)
    - [macaroni and cheese](dish)
    - [pasta primavera](dish)

- intent: inform_ingredients
  examples: |
    - I have [tomatoes](ingredient)
    - I want to use [garlic](ingredient)
    - I've got [pasta](ingredient)
    - I have [cheese](ingredient) and [eggs](ingredient)
    - Can I use [cream](ingredient)?
    - I want to cook with [mushrooms](ingredient)
    - I have [spinach](ingredient) and [ricotta](ingredient)

- intent: inform_dietary_restrictions
  examples: |
    - I'm [vegetarian](dietary_restriction)
    - I'm [vegan](dietary_restriction)
    - I'm [gluten-free](dietary_restriction)
    - I can't eat [dairy](dietary_restriction)
    - I'm allergic to [nuts](dietary_restriction)
    - No [eggs](dietary_restriction) please
''')

### Configuring rasa stories

In [10]:
# Create stories
with open('data/stories.yml', 'w') as f:
    f.write('''
version: "3.1"

stories:
- story: ask for recipe
  steps:
  - intent: greet
  - action: utter_greet
  - intent: ask_recipe
    entities:
    - dish: "pasta"
  - slot_was_set:
    - dish: "pasta"
  - action: action_provide_recipe
  - intent: goodbye
  - action: utter_goodbye

- story: ask for ingredients
  steps:
  - intent: greet
  - action: utter_greet
  - intent: ask_ingredients
    entities:
    - dish: "pasta"
  - slot_was_set:
    - dish: "pasta"
  - action: action_list_ingredients
  - intent: goodbye
  - action: utter_goodbye

- story: ask for cooking steps
  steps:
  - intent: greet
  - action: utter_greet
  - intent: ask_cooking_steps
    entities:
    - dish: "pasta"
  - slot_was_set:
    - dish: "pasta"
  - action: action_explain_steps
  - intent: goodbye
  - action: utter_goodbye

- story: specify dish then ask for recipe
  steps:
  - intent: greet
  - action: utter_greet
  - intent: inform_dish
    entities:
    - dish: "pasta"
  - slot_was_set:
    - dish: "pasta"
  - intent: ask_recipe
  - action: action_provide_recipe
  - intent: goodbye
  - action: utter_goodbye
''')

### Custom actions:

In [12]:
# Create custom actions

# Create the actions directory if it doesn't exist
!mkdir -p actions

with open('actions/actions.py', 'w') as f:
    f.write('''
from typing import Any, Text, Dict, List
from rasa_sdk import Action, Tracker
from rasa_sdk.executor import CollectingDispatcher
from rasa_sdk.events import SlotSet
import pandas as pd
import random
import spacy

# Load spaCy model
nlp = spacy.load("en_core_web_md")

# Load recipe data
recipes_df = pd.read_csv('../pasta_recipes.csv')

class ActionProvideRecipe(Action):
    def name(self) -> Text:
        return "action_provide_recipe"

    def run(self, dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:

        dish = tracker.get_slot("dish")
        dietary_restriction = tracker.get_slot("dietary_restriction")

        if not dish:
            dispatcher.utter_message(text="What dish would you like to make?")
            return []

        # Use spaCy for semantic similarity to find closest match
        dish_vector = nlp(dish).vector

        # Create vectors for all recipe names
        recipe_vectors = []
        for name in recipes_df['name']:
            try:
                recipe_vectors.append((name, nlp(name).vector))
            except:
                continue

        # Find most similar recipe
        best_match = None
        highest_similarity = -1

        for name, vector in recipe_vectors:
            similarity = nlp(dish).similarity(nlp(name))
            if similarity > highest_similarity:
                highest_similarity = similarity
                best_match = name

        if highest_similarity < 0.5:
            dispatcher.utter_message(text=f"I'm sorry, I couldn't find a recipe for {dish}. Would you like to try something else?")
            return []

        # Find the recipe
        recipe = recipes_df[recipes_df['name'] == best_match].iloc[0]

        # Check dietary restrictions
        if dietary_restriction:
            # This is a simplified check - in a real system, you'd want more sophisticated ingredient analysis
            if dietary_restriction == "vegetarian" and any(ing for ing in recipe['ingredients'] if "meat" in ing.lower() or "chicken" in ing.lower() or "beef" in ing.lower()):
                dispatcher.utter_message(text=f"I found a recipe for {best_match}, but it contains meat. Would you like me to find a vegetarian alternative?")
                return []

        # Format the recipe
        ingredients_list = "\\n".join([f"- {ing}" for ing in recipe['ingredients']])
        steps_list = "\\n".join([f"{i+1}. {step}" for i, step in enumerate(recipe['steps'])])

        response = f"Here's a recipe for {best_match}:\\n\\n"
        response += f"Cooking time: {recipe['minutes']} minutes\\n\\n"
        response += f"Ingredients:\\n{ingredients_list}\\n\\n"
        response += f"Instructions:\\n{steps_list}"

        dispatcher.utter_message(text=response)
        return [SlotSet("dish", best_match)]

class ActionListIngredients(Action):
    def name(self) -> Text:
        return "action_list_ingredients"

    def run(self, dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:

        dish = tracker.get_slot("dish")

        if not dish:
            dispatcher.utter_message(text="What dish would you like to know ingredients for?")
            return []

        # Use spaCy for semantic similarity to find closest match
        dish_vector = nlp(dish).vector

        # Create vectors for all recipe names
        recipe_vectors = []
        for name in recipes_df['name']:
            try:
                recipe_vectors.append((name, nlp(name).vector))
            except:
                continue

        # Find most similar recipe
        best_match = None
        highest_similarity = -1

        for name, vector in recipe_vectors:
            similarity = nlp(dish).similarity(nlp(name))
            if similarity > highest_similarity:
                highest_similarity = similarity
                best_match = name

        if highest_similarity < 0.5:
            dispatcher.utter_message(text=f"I'm sorry, I couldn't find ingredients for {dish}. Would you like to try something else?")
            return []

        # Find the recipe
        recipe = recipes_df[recipes_df['name'] == best_match].iloc[0]

        # Format the ingredients list
        ingredients_list = "\\n".join([f"- {ing}" for ing in recipe['ingredients']])

        response = f"Here are the ingredients you'll need for {best_match}:\\n\\n"
        response += ingredients_list

        dispatcher.utter_message(text=response)
        return [SlotSet("dish", best_match)]

class ActionExplainSteps(Action):
    def name(self) -> Text:
        return "action_explain_steps"

    def run(self, dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:

        dish = tracker.get_slot("dish")

        if not dish:
            dispatcher.utter_message(text="What dish would you like to know how to make?")
            return []

        # Use spaCy for semantic similarity to find closest match
        dish_vector = nlp(dish).vector

        # Create vectors for all recipe names
        recipe_vectors = []
        for name in recipes_df['name']:
            try:
                recipe_vectors.append((name, nlp(name).vector))
            except:
                continue

        # Find most similar recipe
        best_match = None
        highest_similarity = -1

        for name, vector in recipe_vectors:
            similarity = nlp(dish).similarity(nlp(name))
            if similarity > highest_similarity:
                highest_similarity = similarity
                best_match = name

        if highest_similarity < 0.5:
            dispatcher.utter_message(text=f"I'm sorry, I couldn't find instructions for {dish}. Would you like to try something else?")
            return []

        # Find the recipe
        recipe = recipes_df[recipes_df['name'] == best_match].iloc[0]

        # Format the steps list
        steps_list = "\\n".join([f"{i+1}. {step}" for i, step in enumerate(recipe['steps'])])

        response = f"Here's how to make {best_match}:\\n\\n"
        response += steps_list

        dispatcher.utter_message(text=response)
        return [SlotSet("dish", best_match)]

class ActionSuggestAlternatives(Action):
    def name(self) -> Text:
        return "action_suggest_alternatives"

    def run(self, dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:

        dish = tracker.get_slot("dish")
        dietary_restriction = tracker.get_slot("dietary_restriction")

        if not dish:
            dispatcher.utter_message(text="What dish would you like alternatives for?")
            return []

        # Get some random pasta recipes
        random_recipes = recipes_df.sample(3)

        response = f"Here are some alternatives you might like:\\n\\n"

        for _, recipe in random_recipes.iterrows():
            response += f"- {recipe['name']}\\n"

        dispatcher.utter_message(text=response)
        return []
''')

### Training :

In [ ]:
# Train the model
!rasa train

# Test the model with shell
# Note: This will be interactive, so you can stop it by pressing Ctrl+C
# !rasa shell --enable-api --cors "*"

### Sample testing for a dish(pasta)

In [17]:
# Create a more advanced recipe generator using spaCy
# Create a more advanced recipe generator using spaCy
import spacy
import random

def generate_recipe(dish_type, ingredients=None, dietary_restrictions=None):
    """
    Generate a recipe based on dish type and optional ingredients/restrictions
    """
    # Load the recipe data - use the correct path
    recipes_df = pd.read_csv('../pasta_recipes.csv')

    # Filter by dish type using spaCy similarity
    nlp = spacy.load("en_core_web_md")
    dish_vector = nlp(dish_type).vector

    # Create vectors for all recipe names
    similarities = []
    for idx, row in recipes_df.iterrows():
        try:
            name_vector = nlp(row['name']).vector
            similarity = nlp(dish_type).similarity(nlp(row['name']))
            similarities.append((idx, similarity))
        except:
            continue

    # Sort by similarity
    similarities.sort(key=lambda x: x[1], reverse=True)

    # Get top 5 most similar recipes
    top_recipes = [recipes_df.iloc[idx] for idx, sim in similarities[:5]]

    # If ingredients are specified, prioritize recipes with those ingredients
    if ingredients:
        ingredient_vectors = [nlp(ing).vector for ing in ingredients]

        # Score recipes based on ingredient matches
        recipe_scores = []
        for recipe in top_recipes:
            score = 0
            recipe_ingredients = recipe['ingredients']

            for user_ing in ingredients:
                user_ing_vector = nlp(user_ing).vector
                best_match = 0

                for recipe_ing in recipe_ingredients:
                    try:
                        recipe_ing_vector = nlp(recipe_ing).vector
                        similarity = nlp(user_ing).similarity(nlp(recipe_ing))
                        if similarity > best_match:
                            best_match = similarity
                    except:
                        continue

                score += best_match

            recipe_scores.append((recipe, score))

        # Sort by score
        recipe_scores.sort(key=lambda x: x[1], reverse=True)
        selected_recipe = recipe_scores[0][0] if recipe_scores else top_recipes[0]
    else:
        selected_recipe = top_recipes[0]

    # Apply dietary restrictions if specified
    if dietary_restrictions:
        # This is a simplified check - in a real system, you'd want more sophisticated ingredient analysis
        restricted_ingredients = []
        if "vegetarian" in dietary_restrictions:
            restricted_ingredients.extend(["meat", "chicken", "beef", "pork"])
        if "vegan" in dietary_restrictions:
            restricted_ingredients.extend(["meat", "chicken", "beef", "pork", "cheese", "egg", "milk", "cream"])
        if "gluten-free" in dietary_restrictions:
            restricted_ingredients.extend(["flour", "wheat", "pasta"])

        # Check if recipe contains restricted ingredients
        contains_restricted = False
        for ing in selected_recipe['ingredients']:
            if any(r_ing in ing.lower() for r_ing in restricted_ingredients):
                contains_restricted = True
                break

        if contains_restricted:
            # Find an alternative recipe
            for recipe in top_recipes[1:]:
                contains_restricted = False
                for ing in recipe['ingredients']:
                    if any(r_ing in ing.lower() for r_ing in restricted_ingredients):
                        contains_restricted = True
                        break

                if not contains_restricted:
                    selected_recipe = recipe
                    break

    # Format the recipe
    ingredients_list = "\n".join([f"- {ing}" for ing in selected_recipe['ingredients']])
    steps_list = "\n".join([f"{i+1}. {step}" for i, step in enumerate(selected_recipe['steps'])])

    recipe_text = f"Recipe for {selected_recipe['name']}\n\n"
    recipe_text += f"Cooking time: {selected_recipe['minutes']} minutes\n\n"
    recipe_text += f"Ingredients:\n{ingredients_list}\n\n"
    recipe_text += f"Instructions:\n{steps_list}"

    return recipe_text

# Test the function
test_recipe = generate_recipe("pasta carbonara", ["eggs", "bacon"])
print(test_recipe)

<ipython-input-17-c4aa7d11fad3>:22: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity = nlp(dish_type).similarity(nlp(row['name']))
<ipython-input-17-c4aa7d11fad3>:50: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity = nlp(user_ing).similarity(nlp(recipe_ing))


Recipe for pasta carbonara

Cooking time: 25 minutes

Ingredients:
- [
- '
- n
- o
- o
- d
- l
- e
- s
- '
- ,
-  
- '
- o
- i
- l
- '
- ,
-  
- '
- b
- a
- c
- o
- n
- '
- ,
-  
- '
- b
- u
- t
- t
- o
- n
-  
- m
- u
- s
- h
- r
- o
- o
- m
- s
- '
- ,
-  
- '
- e
- g
- g
- s
- '
- ,
-  
- '
- l
- i
- g
- h
- t
-  
- c
- r
- e
- a
- m
- '
- ,
-  
- '
- s
- a
- l
- t
- '
- ,
-  
- '
- p
- e
- p
- p
- e
- r
- '
- ,
-  
- '
- p
- a
- r
- m
- e
- s
- a
- n
-  
- c
- h
- e
- e
- s
- e
- '
- ,
-  
- '
- b
- a
- s
- i
- l
- '
- ]

Instructions:
1. [
2. '
3. c
4. o
5. o
6. k
7.  
8. t
9. h
10. e
11.  
12. n
13. o
14. o
15. d
16. l
17. e
18. s
19.  
20. i
21. n
22.  
23. a
24.  
25. p
26. a
27. n
28.  
29. o
30. f
31.  
32. b
33. o
34. i
35. l
36. i
37. n
38. g
39.  
40. w
41. a
42. t
43. e
44. r
45.  
46. ,
47.  
48. f
49. o
50. r
51.  
52. 6
53. -
54. 8
55.  
56. m
57. i
58. n
59. u
60. t
61. e
62. s
63.  
64. o
65. r
66.  
67. u
68. n
69. t
70. i
71. l
72.  
73. a
74. l
75.  
76. d
77. e
7

### A basic UI for user inputs and result of the dishes:


> Result for the input dish's output by trained model can be verified from the dataset




In [21]:
def demo_recipe_bot():
    """
    Simple demo function to test the recipe generator
    """
    print("Welcome to the Recipe Bot!")
    print("What would you like to cook today?")

    dish = input("> ")

    print("\nDo you have any specific ingredients you'd like to use? (comma-separated or 'none')")
    ingredients_input = input("> ")
    ingredients = [ing.strip() for ing in ingredients_input.split(',')] if ingredients_input.lower() != 'none' else None

    print("\nAny dietary restrictions? (comma-separated or 'none')")
    restrictions_input = input("> ")
    restrictions = [rest.strip() for rest in restrictions_input.split(',')] if restrictions_input.lower() != 'none' else None

    recipe = generate_recipe(dish, ingredients, restrictions)

    print("\nHere's your recipe:")
    print(recipe)

# Uncomment to run the demo
demo_recipe_bot()

Welcome to the Recipe Bot!
What would you like to cook today?
> mexican pasta

Do you have any specific ingredients you'd like to use? (comma-separated or 'none')
> basil

Any dietary restrictions? (comma-separated or 'none')
> medium cooked


<ipython-input-17-c4aa7d11fad3>:22: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity = nlp(dish_type).similarity(nlp(row['name']))
<ipython-input-17-c4aa7d11fad3>:50: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity = nlp(user_ing).similarity(nlp(recipe_ing))



Here's your recipe:
Recipe for mexican pasta

Cooking time: 25 minutes

Ingredients:
- [
- '
- f
- e
- t
- t
- u
- c
- c
- i
- n
- e
-  
- p
- a
- s
- t
- a
- '
- ,
-  
- '
- c
- h
- e
- d
- d
- a
- r
-  
- c
- h
- e
- e
- s
- e
-  
- s
- o
- u
- p
- '
- ,
-  
- '
- m
- i
- l
- k
- '
- ,
-  
- '
- p
- i
- c
- a
- n
- t
- e
-  
- s
- a
- u
- c
- e
- '
- ,
-  
- '
- b
- l
- a
- c
- k
-  
- o
- l
- i
- v
- e
- s
- '
- ,
-  
- '
- j
- a
- l
- a
- p
- e
- n
- o
- '
- ]

Instructions:
1. [
2. '
3. b
4. o
5. i
6. l
7.  
8. p
9. a
10. s
11. t
12. a
13.  
14. a
15. s
16.  
17. d
18. i
19. r
20. e
21. c
22. t
23. e
24. d
25.  
26. o
27. n
28.  
29. p
30. a
31. c
32. k
33. a
34. g
35. e
36. '
37. ,
38.  
39. '
40. d
41. r
42. a
43. i
44. n
45.  
46. a
47. n
48. d
49.  
50. p
51. u
52. t
53.  
54. i
55. n
56.  
57. l
58. a
59. r
60. g
61. e
62.  
63. s
64. e
65. r
66. v
67. i
68. n
69. g
70.  
71. b
72. o
73. w
74. l
75. '
76. ,
77.  
78. '
79. m
80. i
81. x
82.  
83. p
84. i
85. c
86. a
87. n
88